In [ ]:
## 이거는 Bing 검색하고 사진 다운로드 받는 코드입니다.


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

keyword = input()

# 웹 접속 - 네이버 이미지 접속
print("접속중")
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(30)

url = 'https://www.bing.com/images/search?q={}&go=%EA%B2%80%EC%83%89&qs=ds&form=QBIR&scope=images&qft=+filterui%3Aface-face'.format(keyword)
driver.get(url)

#페이지 스크롤 다운
body = driver.find_element_by_css_selector('body')
for i in range(21):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

#이미지 링크 수집
imgs = driver.find_elements_by_class_name('mimg')
result = []
count = 1
for img in tqdm(imgs):
    if 'http' in img.get_attribute('src'):
        result.append(img.get_attribute('src'))
        count += 1
    if(count>=400):
        break

driver.close()
print("수집완료")

#폴더생성
print("폴더생성")
import os
if not os.path.isdir('./{}'.format(keyword)):
    os.mkdir('./{}'.format(keyword))

#다운로드
print("다운로드")
from urllib.request import urlretrieve
for index , link in tqdm(enumerate(result)):
    start = link.rfind('.')
    end = link.rfind('&')
    # print(link[start:end])
    filetype = '.jpg' #.png

    urlretrieve(link , './{}/{}{}'.format(keyword,index,filetype))


print("다운로드 완료")

#압축 - 메일
import zipfile
zip_file = zipfile.ZipFile('./{}.zip'.format(keyword),'w')

# print(os.listdir('./{}'.format(keyword)))
for image_gs in os.listdir('./{}'.format(keyword)):
    print(image,"압축파일에 추가중")
    zip_file.write('./{}/{}'.format(keyword,image_gs), compress_type=zipfile.ZIP_DEFLATED)
zip_file.close()
print("압축완료")